### flatten nested json in spark



In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
df1=spark.read.format('json')\
             .option('inferschema',True)\
             .option('multilines',True)\
             .option('header',True).load('resturant_json_data.json')

In [5]:
df1.show(5)

+----+-------+--------------------+-------------+-------------+-------------+------+
|code|message|         restaurants|results_found|results_shown|results_start|status|
+----+-------+--------------------+-------------+-------------+-------------+------+
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17066603}, b9...|         6835|           20|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|                  []|            0|            0|            1|  null|
|null|   null|[{{{17093124}, b9...|         8680|           20|            1|  null|
+----+-------+--------------------+-------------+-------------+-------------+------+
only showing top 5 rows


In [6]:
df1.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [9]:
df1.select("*",explode("restaurants").alias("new_restro")).printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i

In [10]:
df1.select("*",explode("restaurants").alias("new_restro"))\
   .drop("restaurants").printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- new_restro: struct (nullable = true)
 |    |-- restaurant: struct (nullable = true)
 |    |    |-- R: struct (nullable = true)
 |    |    |    |-- res_id: long (nullable = true)
 |    |    |-- apikey: string (nullable = true)
 |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |-- cuisines: string (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- deeplink: string (nullable = true)
 |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- events_url: string (nullable = true)
 |    |    |-- featured_image: string (nullable = true)
 |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |-- has_table_bo

In [11]:
df1.select("*",explode("restaurants").alias("new_restro"))\
   .drop("restaurants")\
   .select("new_restro.restaurant.R.res_id").show()

+--------+
|  res_id|
+--------+
|17066603|
|17059541|
|17064405|
|17057797|
|17057591|
|17064266|
|17060516|
|17060320|
|17059060|
|17059012|
|17060869|
|17061231|
|17058534|
|17057925|
|17064031|
|17061237|
|17061253|
|17061296|
|17061205|
|17057397|
+--------+
only showing top 20 rows


In [19]:
df1.select("*",explode("restaurants").alias("new_restro"))\
   .drop("restaurants")\
   .select("*","new_restro.restaurant.R.res_id").show(5)

+----+-------+-------------+-------------+-------------+------+--------------------+--------+
|code|message|results_found|results_shown|results_start|status|          new_restro|  res_id|
+----+-------+-------------+-------------+-------------+------+--------------------+--------+
|null|   null|         6835|           20|            1|  null|{{{17066603}, b90...|17066603|
|null|   null|         6835|           20|            1|  null|{{{17059541}, b90...|17059541|
|null|   null|         6835|           20|            1|  null|{{{17064405}, b90...|17064405|
|null|   null|         6835|           20|            1|  null|{{{17057797}, b90...|17057797|
|null|   null|         6835|           20|            1|  null|{{{17057591}, b90...|17057591|
+----+-------+-------------+-------------+-------------+------+--------------------+--------+
only showing top 5 rows


In [24]:
df1.select("*",explode("restaurants").alias("new_restro"))\
   .drop("restaurants")\
   .select("*","new_restro.restaurant.R.res_id",
            explode("new_restro.restaurant.establishment_types").alias("establishment_types_new"),
          "new_restro.restaurant.name").drop("new_restro").printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- res_id: long (nullable = true)
 |-- establishment_types_new: string (nullable = true)
 |-- name: string (nullable = true)


AttributeError: ("'NoneType' object has no attribute 'show'",)

In [25]:
df1.select("*",explode("restaurants").alias("new_restro"))\
   .drop("restaurants")\
   .select("*","new_restro.restaurant.R.res_id",
            explode("new_restro.restaurant.establishment_types").alias("establishment_types_new"),
          "new_restro.restaurant.name").drop("new_restro").show()

+----+-------+-------------+-------------+-------------+------+------+-----------------------+----+
|code|message|results_found|results_shown|results_start|status|res_id|establishment_types_new|name|
+----+-------+-------------+-------------+-------------+------+------+-----------------------+----+
+----+-------+-------------+-------------+-------------+------+------+-----------------------+----+


In [27]:
df1.select("*",explode("restaurants").alias("new_restro"))\
   .drop("restaurants")\
   .select("*","new_restro.restaurant.R.res_id",
            explode_outer("new_restro.restaurant.establishment_types").alias("establishment_types_new"),
          "new_restro.restaurant.name").drop("new_restro").show(truncate=False)

+----+-------+-------------+-------------+-------------+------+--------+-----------------------+------------------------------------+
|code|message|results_found|results_shown|results_start|status|res_id  |establishment_types_new|name                                |
+----+-------+-------------+-------------+-------------+------+--------+-----------------------+------------------------------------+
|null|null   |6835         |20           |1            |null  |17066603|null                   |The Coop                            |
|null|null   |6835         |20           |1            |null  |17059541|null                   |Maggiano's Little Italy             |
|null|null   |6835         |20           |1            |null  |17064405|null                   |Tako Cheena by Pom Pom              |
|null|null   |6835         |20           |1            |null  |17057797|null                   |Bosphorous Turkish Cuisine          |
|null|null   |6835         |20           |1            |null  